# Objectives

 1. Multi-varaite tests: The GRS test
 


In [2]:
# First we start loading our favourite packages with the names we used to
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
# import regression package
import statsmodels.api as sm

In [3]:
# import market factor
url = "https://www.dropbox.com/s/9346pp2iu5prv8s/MonthlyFactors.csv?dl=1"
Factors = pd.read_csv(url,index_col=0, 
                         parse_dates=True,na_values=-99)
Factors=Factors
# keep only market and RF
Factors=Factors.iloc[:,0:2]
Factors.head()

# test assets

# import BEME (Book-equity over Market equity) sorted portfolios
# BEME 1 has low book to market- > Known as growth stocks (example: Tesla, Amazon, Facebook)

# BEME 10 has high book to market -> Known as value stocks ( old firms: utilities, banks, insurance companies)

# but it changes....

url = "https://www.dropbox.com/s/gds8ho9rt1imw03/Valueportfolios.csv?dl=1"
parser = lambda date: pd.to_datetime(date,format='%Y%m')
Data = pd.read_csv(url,index_col=0, 
                         parse_dates=True,date_parser=parser,skiprows=23,nrows=1092,na_values=-99.99)

print(Data.tail())
# neeed to adjsut the date to end of month convention (Even thugh both returns for a given month are end-of-month to end of month,
# when we importer the value portfolios python used as date the first day of the month . To fix that, we will convert it to end of month
# It is important to do that before the merge. Why? )
Data.index=Data.index+pd.tseries.offsets.MonthEnd()
print(Data.head())
# ready to merge with Market and RF

Data=pd.merge(Data,Factors,left_index=True,right_index=True)
Data.head()

# excess returns

# transform in excess returns
Data.iloc[:,0:10]=Data.iloc[:,0:10].subtract(Data['RF'],axis=0)
Data.iloc[:,11]=Data.iloc[:,11].subtract(Data['RF'],axis=0)
# take the risk-free rate column out of the database
del Data['RF']
Data.head()

            BEME1  BEME2  BEME3  BEME4  BEME5  BEME6  BEME7  BEME8  BEME9  \
2017-02-01   4.29   5.22   3.52   2.47   3.90   0.96   2.03   3.24   3.38   
2017-03-01   0.97   2.04   0.30   0.31   0.07  -0.70  -0.99  -1.22  -3.09   
2017-04-01   2.44   1.32   2.16   1.39   2.43  -0.52  -0.44  -0.56  -0.64   
2017-05-01   2.59   2.86   3.10  -0.23   1.29  -0.33  -1.26  -0.70  -2.44   
2017-06-01   0.29  -0.47   0.25   1.15  -0.69   1.21   1.67   2.57   4.31   

            BEME10  
2017-02-01    5.03  
2017-03-01   -2.28  
2017-04-01   -0.86  
2017-05-01   -2.68  
2017-06-01    5.70  
            BEME1  BEME2  BEME3  BEME4  BEME5  BEME6  BEME7  BEME8  BEME9  \
1926-07-31   4.63   1.24  11.40   1.68   1.50   2.09   2.73   1.58   0.63   
1926-08-31   0.15   2.18   6.40   2.38   1.58   2.63   4.46   4.94   8.31   
1926-09-30  -0.11  -2.30   6.32   0.73  -0.89   0.38  -0.28   0.17  -2.16   
1926-10-31  -3.78  -1.63  -5.01  -1.41  -2.31  -2.10  -4.65  -2.80  -5.46   
1926-11-30   6.03   1.49  

,BEME1,BEME2,BEME3,BEME4,BEME5,BEME6,BEME7,BEME8,BEME9,BEME10,MKT
1926-07-31,4.41,1.02,11.18,1.46,1.28,1.87,2.51,1.36,0.41,0.30,2.96
1926-08-31,-0.10,1.93,6.15,2.13,1.33,2.38,4.21,4.69,8.06,3.67,2.64
1926-09-30,-0.34,-2.53,6.09,0.50,-1.12,0.15,-0.51,-0.06,-2.39,0.21,0.36
1926-10-31,-4.10,-1.95,-5.33,-1.73,-2.63,-2.42,-4.97,-3.12,-5.78,-1.49,-3.24
1926-11-30,5.72,1.18,0.80,3.32,1.66,1.49,3.69,3.54,2.46,1.45,2.53




# Multivariate time-series test


But this approach tests the model "Asset by Asset": We tested if there is any single portfolio that is inconsistent with the model.

But we can get much more power by using all the assets at the same time

- What if no single asset has a statisically significant alpha, could them together reject the model?

- Joining all the small SR can give a much higher overall SR and much stronger rejection of the model.

- This will also be relevant for investing. You want to invest on the overall portfolio has the higher SR and it is unlikely that it will be the individual portfolios. As long they are not perfectly correlated you will likely get something from combining them


### How to test the model using all the assets at the same time?

#### The answer is the GRS test

Run Time-series regression asset by asset

$$R_i^e =\alpha_i+\beta_{i,M} R_{M}^e+\epsilon_i$$

stack the alphas in a vector $\hat{\alpha}$

If you have N test assets you will have N time-series of residuals $\epsilon_i$

Use this to estimate the variance-covariance matrix of the assets  residual risk

$$\hat{\Sigma}_{\epsilon}=var([\epsilon_1,...,\epsilon_1])$$

This yield a N by N covariance matrix $\hat{\Sigma}_{\epsilon}$

$$T\left(1+\left(\frac{average(R^e_M)}{std(R^E_M)}\right)^2\right)^{-1}\hat{\alpha}\hat{\Sigma}_{\epsilon}^{-1}\hat{\alpha}' \sim Chi2(N)$$

In [5]:
# creaty copies of the dataset
df=Data.copy()
Resid=Data.copy()

# Create the right hand side variable for the regression:
# vector os 1's (for the intercept) plus returns on the market
x= sm.add_constant(df['MKT'])
# wea re going to use Resid to store the residuals of the regresion, so we are deleting the market column since we will
# only run regressions of the differetn portfolio on the market
del Resid['MKT']
# here we are just initiatinng a data frame where we will store the alpha of each portfolio
# so the preside operation that I am doing below, does not matter as long it returns a dataframe where the 
#columns are the portfolio names
E=pd.DataFrame(Resid.mean())
E.columns=['alpha']

#now I am going to loop through the portfolios and run a time-series regression for each just as before
for portfolios in E.index:
    y= df[portfolios]
    results= sm.OLS(y,x).fit()
    # We are storing the alphas
    E.at[portfolios,'alpha']=results.params[0]
    # we are stroing the ENTIRE time-seires of regression residuals
    Resid.loc[:,portfolios]=results.resid

# We are now ready to perform the GRS test

# what information do we need?




$$T\left(1+\left(\frac{average(R^e_M)}{std(R^E_M)}\right)^2\right)^{-1}\hat{\alpha}\hat{\Sigma}_{\epsilon}^{-1}\hat{\alpha}' \sim Chi2(N)$$

In [6]:
# first the market moments
avgMkt=df.MKT.mean()
stdMkt=df.MKT.std()
# the size of the sample
T=df.shape[0]
# the inverse of the residual covariance matrix
invCov=np.linalg.inv(Resid.cov())
# the chi statistic!
chi=T*(1+(avgMkt/stdMkt)**2)**(-1)*(E.alpha.values @invCov@ E.alpha.values.T)
chi

20.734452742206294

1. We need to compare this Chi2 statistic to the critical values of the Chi2 distribution with N degrees of freedom

In [7]:
from scipy.stats import chi2

In [8]:
# choose degress of freedom (number of test assets)
dfreedom=10
# critical threshold for 10%, 5%, 1%, 0.1%
print([chi2.ppf(1-0.1,dfreedom),chi2.ppf(1-0.05,dfreedom),chi2.ppf(1-0.01,dfreedom),chi2.ppf(1-0.001,dfreedom)])

# we can also look for the equivalent P-value

1-chi2.cdf(chi,dfreedom)


[15.987179172105263, 18.307038053275146, 23.209251158954356, 29.58829844507442]


0.02302325955319673

We reject with about 98% confidence.

There is a 2.3% chance that this abnormal performance of Book-to-Market sorted portfolios were a statistical fluke 

In [10]:
# the GRS test

def GRS(df,beg,end):
    df=df[beg:end]
    Resid=df.copy()
    x= sm.add_constant(df['MKT'])
    del Resid['MKT']
    E=pd.DataFrame(Resid.mean())
    E.columns=['alpha']
    for portfolios in E.index:
        y= df[portfolios]
        results= sm.OLS(y,x).fit()
        E.at[portfolios,'alpha']=results.params[0]
        Resid.loc[:,portfolios]=results.resid
     
        
    avgMkt=df.MKT.mean()
    stdMkt=df.MKT.std()
    T=df.shape[0]
    invCov=np.linalg.inv(Resid.cov())
    chi=T*(1+(avgMkt/stdMkt)**2)**(-1)*(E.alpha.values @invCov@ E.alpha.values.T)
    N=Resid.shape[1]
    return [chi,(1-chi2.cdf(chi,N))*100]

In [11]:
chi=GRS(Data,'1963','2016')
chi


[23.31578017926853, 0.9639119916886818]

In [ ]:
chi=GRS(Data,'1926','1963')
chi

In [ ]:
chi=GRS(Data,'1963','1992')
chi